In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from SALib import ProblemSpec
import contextlib




In [2]:
# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 2000
# Number of parameters
n_params = 22

# Read Input Data
df_x = pd.read_csv(f'../Emulation/Input/input_{n_samples}_{n_params}params.csv')

# Select relevant inputs only
relevant_columns = []
for col in df_x.columns:
    relevant_columns.append(col)
    if col == 'T': break

# Select only first relevant inputs 
filtered_input = df_x[relevant_columns]

# Import Emulators
emulators = pd.read_pickle(f"../Emulation/Outputs/Output_{n_samples}_{n_params}params/Emulators/linear_models_and_r2_scores_{n_samples}.csv")

# Extract the emulator names
emulator_list = emulators.index.to_list()

# Directory to save results
output_dir = "Sensitivity_Analysis_Results"
os.makedirs(output_dir, exist_ok=True)


In [4]:
emulators

,R2_Score,MSE,RSE,Model
0,0.726457,1.105458,1.057506,LinearRegression()
1,0.714323,0.762935,0.878528,LinearRegression()
2,0.698990,0.541008,0.739798,LinearRegression()
3,0.679376,0.403680,0.639043,LinearRegression()
4,0.655743,0.322777,0.571430,LinearRegression()
...,...,...,...,...
PC6,0.259290,0.043770,0.210425,LinearRegression()
PC7,0.461797,0.020073,0.142502,LinearRegression()
PC8,0.223073,0.006045,0.078202,LinearRegression()
PC9,0.063827,0.004280,0.065798,LinearRegression()


In [5]:
# Iterate over each emulator
for emulator_name in emulator_list:
    print(f"Processing {emulator_name}...")

    # Define problem spec for sensitivity analysis
    problem = ProblemSpec({
        'num_vars': len(relevant_columns),
        'names': relevant_columns,
        'bounds': filtered_input[relevant_columns].describe().loc[['min', 'max']].T.values,
        "outputs": [emulator_name],
    })

    linear_model = emulators.loc[emulator_name, 'Model']

    # Sample inputs
    problem.sample_sobol(1024)
    X_samples = problem.samples

    #print(f"Samples: {X_samples.shape}")

    
    # Extract emulator coefficients and intercept
    beta_matrix = np.array(linear_model.coef_)  # (num_outputs, num_vars) or (num_vars,)
    intercept = np.array(linear_model.intercept_)  # (num_outputs,) or scalar

    # Ensure correct shape
    if beta_matrix.ndim == 1:
        beta_matrix = beta_matrix.reshape(1, -1)  # Convert (num_vars,) → (1, num_vars)
    if intercept.ndim == 0:
        intercept = np.array([intercept])  # Convert scalar → (1,)
    
    #print(f"beta_matrix: {beta_matrix.shape}")
    #print(f"intercept: {intercept.shape}")

    # Compute emulator outputs
    Y_samples = X_samples @ beta_matrix.T + intercept  # (num_samples, num_outputs)

    #print(f"Y_samples shape: {Y_samples.shape}")

    # Flatten if necessary
    Y_reshape = Y_samples.reshape(-1)
    #print(f"Y_reshape shape: {Y_reshape.shape}")

    # Set and analyze results
    problem.set_results(Y_reshape)
    sobol_indices = problem.analyze_sobol(print_to_console=True)

    # Sort results
    total, first, second = sobol_indices.to_df()
    total.sort_values('ST', inplace=True, ascending=False)

    # Save results
    result_data = pd.DataFrame({
        "Parameter": total.index,
        "ST": total['ST'],
        "ST_conf": total['ST_conf']
    })
    safe_emulator_name = emulator_name.replace("/", "_") # Avoid slashes in file names

    result_file = os.path.join(output_dir, f"sensitivity_{safe_emulator_name}.csv")
    result_data.to_csv(result_file, index=False)
    print(f"Saved results for {safe_emulator_name} to {result_file}")

print("All analyses completed.")


Processing 0...


/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                    ST       ST_conf
# svn.c   2.879609e-02  2.437969e-03
pas.r     1.298786e-04  1.064804e-05
pas.c     1.384634e-03  1.131644e-04
pas.l     1.447801e-05  1.282720e-06
pat.r     4.626363e-01  3.481850e-02
pat.c     4.681294e-01  3.855313e-02
pat.l     1.042470e-04  7.791608e-06
pvn.r     7.181524e-04  7.245449e-05
pvn.c     3.813705e-03  3.368095e-04
ao.CQ     3.706650e-08  3.138769e-09
mi.CQ     3.915825e-03  3.177700e-04
po.CQ     1.164710e-04  9.522527e-06
ti.CQ     2.136822e-05  1.856458e-06
rv.E_pas  1.635368e-03  1.561952e-04
rv.E_act  5.649462e-04  4.761166e-05
rv.v_ref  2.964321e-03  2.490780e-04
rv.k_pas  1.445035e-03  1.273294e-04
ra.E_pas  6.952904e-03  5.874178e-04
ra.E_act  1.280842e-03  1.106297e-04
ra.v_ref  9.018392e-04  7.747063e-05
ra.k_pas  1.195431e-02  1.039526e-03
T         2.567738e-03  2.357814e-04
                    S1   S1_conf
# svn.c   2.881692e-02  0.013595
pas.r     1.298838e-04  0.000924
pas.c     1.354345e-03  0.003131
pas.l     6.65185

/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                    ST       ST_conf
# svn.c   3.068197e-02  2.822862e-03
pas.r     1.313794e-04  1.098807e-05
pas.c     1.452251e-03  1.303718e-04
pas.l     1.297826e-05  1.304729e-06
pat.r     4.492284e-01  3.843780e-02
pat.c     4.695150e-01  3.992257e-02
pat.l     7.186668e-05  5.844648e-06
pvn.r     6.763307e-04  6.336057e-05
pvn.c     3.861024e-03  3.676383e-04
ao.CQ     1.552347e-08  1.170537e-09
mi.CQ     3.874238e-03  3.462578e-04
po.CQ     7.448355e-05  7.024914e-06
ti.CQ     3.352831e-06  2.968714e-07
rv.E_pas  4.293275e-03  4.248733e-04
rv.E_act  8.974857e-04  8.785890e-05
rv.v_ref  3.166465e-03  2.719470e-04
rv.k_pas  5.565042e-03  5.099008e-04
ra.E_pas  7.436646e-03  7.593357e-04
ra.E_act  1.451303e-03  1.427769e-04
ra.v_ref  8.743863e-04  7.555574e-05
ra.k_pas  1.220197e-02  1.209485e-03
T         3.614132e-03  3.285242e-04
                    S1   S1_conf
# svn.c   3.069525e-02  0.018204
pas.r     1.506501e-04  0.001060
pas.c     1.590112e-03  0.003458
pas.l     1.32200

...

/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                ST   ST_conf
# svn.c   0.015019  0.001419
pas.r     0.011196  0.001028
pas.c     0.007976  0.000797
pas.l     0.011748  0.001055
pat.r     0.278982  0.021788
pat.c     0.106213  0.009133
pat.l     0.000854  0.000072
pvn.r     0.000313  0.000026
pvn.c     0.016884  0.001621
ao.CQ     0.004135  0.000397
mi.CQ     0.000042  0.000004
po.CQ     0.017717  0.001515
ti.CQ     0.136633  0.011246
rv.E_pas  0.010154  0.000910
rv.E_act  0.036530  0.003369
rv.v_ref  0.015145  0.001298
rv.k_pas  0.275837  0.024595
ra.E_pas  0.003515  0.000311
ra.E_act  0.014038  0.001334
ra.v_ref  0.001578  0.000140
ra.k_pas  0.032844  0.003118
T         0.001971  0.000169
                S1   S1_conf
# svn.c   0.019285  0.011665
pas.r     0.011540  0.009498
pas.c     0.011095  0.007230
pas.l     0.011749  0.009618
pat.r     0.280595  0.037067
pat.c     0.106217  0.025065
pat.l     0.000886  0.002636
pvn.r     0.000333  0.001418
pvn.c     0.016657  0.010336
ao.CQ     0.004133  0.006449
mi.CQ     0.00

/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                ST   ST_conf
# svn.c   0.007654  0.000793
pas.r     0.000577  0.000054
pas.c     0.018172  0.001520
pas.l     0.012062  0.001215
pat.r     0.187872  0.018866
pat.c     0.102352  0.008391
pat.l     0.006140  0.000605
pvn.r     0.000232  0.000019
pvn.c     0.020629  0.001819
ao.CQ     0.014078  0.001163
mi.CQ     0.000122  0.000012
po.CQ     0.050221  0.005035
ti.CQ     0.008831  0.000870
rv.E_pas  0.016118  0.001719
rv.E_act  0.385405  0.034483
rv.v_ref  0.000661  0.000062
rv.k_pas  0.053408  0.005353
ra.E_pas  0.016704  0.001610
ra.E_act  0.061121  0.005370
ra.v_ref  0.004207  0.000386
ra.k_pas  0.005577  0.000493
T         0.022445  0.001919
                S1   S1_conf
# svn.c   0.006842  0.008238
pas.r     0.000502  0.002273
pas.c     0.018181  0.011542
pas.l     0.011414  0.009331
pat.r     0.187460  0.036453
pat.c     0.102193  0.023465
pat.l     0.005566  0.006885
pvn.r     0.000231  0.001201
pvn.c     0.021039  0.013379
ao.CQ     0.013909  0.010449
mi.CQ     0.00

/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
